In [ ]:
from utils import read_lines
from collections import defaultdict

def parse_program(s):
    return [int(x) for x in s.split(',')]

def parse_code(num):
    if num == 99:
        return num, [0]
    else:
        digits = []
        while num > 0:
            digits.append(num % 10)
            num //=10
        digits += [0] * (5-len(digits))
        opcode = digits[0]
        param_modes = digits[2:]
        return opcode, param_modes

def run(program, input=0):
    i = 0
    outputs = []
    extend_memory = defaultdict(int)
    relative_base = 0
    def read_memory(addr):
        if addr < len(program):
            return program[addr]
        else:
            return extend_memory[addr]
        
    def write_memory(idx, value, mode):
        addr = program[idx]
        if mode == 2:
            addr = relative_base + addr
        if addr < len(program):
            program[addr] = value
        else:
            extend_memory[addr] = value
    
    def get_oprand(idx, mode):
        if mode == 1:
            return program[idx]
        if mode == 2:
            addr = relative_base + program[idx]
        else:
            addr = program[idx]
        return read_memory(addr)

    while i < len(program):
        opcode, param_modes = parse_code(program[i])
        if opcode in (1, 2, 5, 6, 7, 8):
            oprand1 = get_oprand(i+1, param_modes[0])
            oprand2 = get_oprand(i+2, param_modes[1])
    
            if opcode == 1:
                write_memory(i+3, oprand1 + oprand2, param_modes[2])
                i+=4
            elif opcode == 2:
                write_memory(i+3, oprand1 * oprand2, param_modes[2])
                i+=4
            elif opcode == 5:
                if oprand1 != 0:
                    i = oprand2
                else:
                    i += 3
            elif opcode == 6:
                if oprand1 == 0:
                    i = oprand2
                else:
                    i += 3
            elif opcode == 7:
                write_memory(i+3, 1 if oprand1 < oprand2 else 0, param_modes[2])
                i+=4
            elif opcode == 8:
                write_memory(i+3, 1 if oprand1 == oprand2 else 0, param_modes[2])
                i+=4
        elif opcode ==3 :
            write_memory(i+1, input, param_modes[0])
            i += 2
        elif opcode == 4:
            oprand1 = get_oprand(i+1, param_modes[0])
            outputs.append(oprand1)
            i += 2
        elif opcode == 9:
            oprand1 = get_oprand(i+1, param_modes[0])
            relative_base += oprand1
            i+=2
        elif program[i] == 99:
            return outputs
        else:
            raise ValueError(f'illegal code {i} {program[i]}')
        
def part1(input_file):
    s = read_lines(input_file)[0]
    program = parse_program(s)
    outputs = run(program)
    ans = 0
    for i in range(2, len(outputs), 3):
        if outputs[i] == 2:
            ans += 1
    return ans


In [ ]:
part1('inputs/day13.txt')

In [73]:
def print_grid(grid):
    for row in grid:
        print(''.join(str(x) for x in row))
    
def run2(program):
    i = 0
    outputs = []
    extend_memory = defaultdict(int)
    relative_base = 0
    def read_memory(addr):
        if addr < len(program):
            return program[addr]
        else:
            return extend_memory[addr]
        
    def write_memory(idx, value, mode):
        addr = program[idx]
        if mode == 2:
            addr = relative_base + addr
        if addr < len(program):
            program[addr] = value
        else:
            extend_memory[addr] = value
    
    def get_oprand(idx, mode):
        if mode == 1:
            return program[idx]
        if mode == 2:
            addr = relative_base + program[idx]
        else:
            addr = program[idx]
        return read_memory(addr)

    grid = [[0] * 37 for _ in range(20)]
    block_count = 0
    ball_positions = []
    paddle_positions = []
    while i < len(program):
        opcode, param_modes = parse_code(program[i])
        if opcode in (1, 2, 5, 6, 7, 8):
            oprand1 = get_oprand(i+1, param_modes[0])
            oprand2 = get_oprand(i+2, param_modes[1])
    
            if opcode == 1:
                write_memory(i+3, oprand1 + oprand2, param_modes[2])
                i+=4
            elif opcode == 2:
                write_memory(i+3, oprand1 * oprand2, param_modes[2])
                i+=4
            elif opcode == 5:
                if oprand1 != 0:
                    i = oprand2
                else:
                    i += 3
            elif opcode == 6:
                if oprand1 == 0:
                    i = oprand2
                else:
                    i += 3
            elif opcode == 7:
                write_memory(i+3, 1 if oprand1 < oprand2 else 0, param_modes[2])
                i+=4
            elif opcode == 8:
                write_memory(i+3, 1 if oprand1 == oprand2 else 0, param_modes[2])
                i+=4
        elif opcode ==3 :
            # print_grid(grid)
            # command = input('acion? ')
            command = 0
            if len(ball_positions) > 1:
                
                delta = ball_positions[-1][0] - ball_positions[-2][0]
                if delta == 0:  # bounce
                    delta = ball_positions[-3][0] - ball_positions[-2][0]
                
                if ball_positions[-1][1] == 17 and ball_positions[-1][0] == paddle_positions[-1][0]:
                    next_x = ball_positions[-1][0]
                else:
                    next_x = ball_positions[-1][0] + delta
                
                if next_x == 0:
                    next_x = 1
                elif next_x == len(grid[0]) - 1:
                    next_x = len(grid[0]) - 2
                if paddle_positions[-1][0] < next_x:
                    command = 1
                elif paddle_positions[-1][0] > next_x:
                    command = -1
                else:
                    if ball_positions[-1][1] == 16:
                        command = 0
                    else:
                        
                        command = 0
                # print(ball_positions[-1], next_x, command, paddle_positions[-1])
            write_memory(i+1, int(command), param_modes[0])
            i += 2
        elif opcode == 4:
            oprand1 = get_oprand(i+1, param_modes[0])
            outputs.append(oprand1)
            if len(outputs) == 3:
                if outputs[0] == -1 and outputs[1] == 0:
                    if block_count == 0:
                        return outputs[2]
                else:
                    if outputs[2] == 2:
                        block_count += 1
                    elif outputs[2] == 3:
                        paddle_positions.append(outputs[:2])
                    elif outputs[2] == 4:
                        ball_positions.append(outputs[:2])
                    elif outputs[2] == 0 and  grid[outputs[1]][outputs[0]] == 2:
                        block_count -= 1
                grid[outputs[1]][outputs[0]] = outputs[2]
                outputs = []
            i += 2
        elif opcode == 9:
            oprand1 = get_oprand(i+1, param_modes[0])
            relative_base += oprand1
            i+=2
        elif program[i] == 99:
            return ball_positions, paddle_positions
        else:
            raise ValueError(f'illegal code {i} {program[i]}')
   
def part2(input_file):
    s = read_lines(input_file)[0]
    program = parse_program(s)
    program[0] = 2
    return run2(program)

part2('inputs/day13.txt')

12779